In [138]:
import torch
import mymodel
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import pickle
import MeCab
from mecab_split import mecabsplit
#tagger = MeCab.Tagger()
tagger = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ko-dic')

In [139]:
content_vocab = {'unk':0}
intent_vocab={}
intent_list=[]

data_intent=''
intent_idx=0
vocab_idx=1

for line in open('mydata.txt','r',encoding='utf-8'):
    line = line.strip().split('\t')
    if len(line)>1:
        intent=line[1]
        if intent not in intent_vocab:
            intent_vocab[intent]=intent_idx
            intent_list.append(intent)
            intent_idx +=1
    else:
        line = mecabsplit(tagger,line[0],False)
        for it in line:
            if it not in content_vocab:
                content_vocab[it] = vocab_idx
                vocab_idx +=1
                
                

In [140]:
cnn = mymodel.CNN_Text(vocab_idx,intent_idx)
print(vocab_idx, intent_idx) # 파일에 나타난 단어수, 의도 갯수(클래스)
optimizer = torch.optim.Adam(cnn.parameters())
cnn.train()

356 4


CNN_Text(
  (embed): Embedding(356, 100)
  (convs1): ModuleList(
    (0): Conv2d(1, 20, kernel_size=(1, 100), stride=(1, 1))
    (1): Conv2d(1, 20, kernel_size=(2, 100), stride=(1, 1))
    (2): Conv2d(1, 20, kernel_size=(3, 100), stride=(1, 1))
  )
  (dropout): Dropout(p=0.2)
  (fc1): Linear(in_features=60, out_features=4, bias=True)
)

In [141]:
ls

IMDB Dataset.csv* main.ipynb*       mydata.txt*       response.txt*
__pycache__/      mecab_split.py*   mymodel.py*       testdata.txt*


In [142]:
epoch = 10
for e in range(epoch):
    totalloss = 0
    for line in open('mydata.txt','r',encoding='utf-8'):
        line = line.strip().split('\t')
        
        if len(line)> 1:
            target = Variable(torch.LongTensor([intent_vocab[line[1]]]))
            continue

        optimizer.zero_grad()

        cont = []
        line = mecabsplit(tagger,line[0],False)
        for it in line:
            cont.append(content_vocab[it])
        cont = Variable(torch.LongTensor(cont)).view(1,-1)
        pred = cnn(cont)
        
        loss = F.cross_entropy(pred,target)
        totalloss += loss.data
        loss.backward()
        optimizer.step()
    print (totalloss)

tensor(221.0006)
tensor(193.2096)
tensor(169.4205)
tensor(149.7665)
tensor(132.2559)
tensor(117.3417)
tensor(109.2918)
tensor(103.2583)
tensor(96.4019)
tensor(90.6367)


In [145]:
response = []
for line in open('response.txt','r',encoding='utf-8'):
    line=line.strip()
    response.append(line)

In [150]:
cnn.eval()
for line in open('testdata.txt','r',encoding='utf-8'):
    line = line.strip()
    
    line = mecabsplit(tagger,line,False)
    cont = []
    for it in line:
        if it in content_vocab:
            cont.append(content_vocab[it]) # cont에는 입력문에 나타난 단어들의 index 저장됨
        else:
            cont.append(content_vocab['unk'])
    cont = Variable(torch.LongTensor(cont)).view(1,-1)
    pred = cnn(cont)
    v,i = torch.max(pred,1) # pred는 (p1, p2) 즉, 클래스별 확률 v: 둘중 큰값 i:큰값 클래스의 인덱스
    
    print('input : ',line)
    print (torch.nn.functional.softmax(pred,dim=-1).data.numpy() )
    print('intent : ',intent_list[int(i)])

input :  ['본인', '은', '무엇', '을', '잘', '하나', '이', '오']
[[0.21940705 0.15997027 0.42057458 0.20004816]]
intent :  ASK_SOCIALITY
input :  ['삼성', '에', '대하', '아서', '어떻게', '생각', '하', '시', '어요']
[[0.05949122 0.6748369  0.2174089  0.04826294]]
intent :  ASK_PERSONALITY
input :  ['어리', 'ᆫ', '시절', '부터', '연락', '하', '는', '친구', '는', '몇', '명', '이', 'ᆫ가요']
[[0.14240685 0.05898617 0.77829105 0.02031595]]
intent :  ASK_SOCIALITY
input :  ['여자', '친구', '있', '어요']
[[0.18112522 0.29110777 0.2847897  0.24297732]]
intent :  ASK_PERSONALITY
input :  ['부모', '님', '에', '대하', '아', '어떻게', '생각', '하', '나요']
[[0.12012345 0.40078163 0.4273098  0.05178505]]
intent :  ASK_SOCIALITY
